# Chapter 11. Machine Learning

## Modeling

A model is a specification of a mathematical or probabilistic relationship that exists between different variables.

## What is Machine Learning?

This book defines [machine learning](https://en.wikipedia.org/wiki/Machine_learning) as creating and using models that are learned from data, which in other contexts might be referred to as [predictive modeling](https://en.wikipedia.org/wiki/Predictive_modelling) or [data mining](https://en.wikipedia.org/wiki/Data_mining).  
Typically, our goal will be to use existing data to develop models that we can use to predict various outcomes for new (incoming) data, such as:  
- Predicting whether an email message is spam or not.
- Predicting whether a credit card transaction is fraudulent or not.
- Predicting which advertisement a shopper is most likely to click on.
- Predicting which football team is going to win the Super Bowl.

We will be looking at:  
- Supervised Models in which there is a set of data labeled with the correct answers to learn from.
- Unsupervised Models in which there are no such labels.  
There are other types of machine learning, such as semisupervised and reinforcement, that will not be covered here.  
There are many different machine learning models that might describe the particular problem we're trying to solve or relationship we're trying to find.  
Throughout the rest of this book we'll be investigating different families of models that we can learn, so let's begin by better understanding the fundamentals of machine learning.

## Overfitting and Underfitting

Overfitting happens when you create a model that performs well on the data you train it on but that generalizes poorly to new data.  
Underfitting happens when you create a model that performs poorly on the training data as well.